<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [45]:
''' Imports '''
''' Is any Data Science notebook complete without pandas and numpy? '''
import pandas as pd
import numpy as np

''' Base stuff '''
import os
import re

''' Gensim Stuff '''
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora 
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis.gensim


In [127]:
''' Brining in the Data '''
df = pd.read_csv('./data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv')
df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [128]:
# Checking out value counts from words  in the review text
df['reviews.text'].value_counts(normalize=True)[:50]

good                0.002118
great               0.001729
Good                0.001482
Great               0.001341
ok                  0.001165
Great price         0.001059
great price         0.000988
Excellent           0.000988
Great value         0.000953
good price          0.000812
great value         0.000777
Great price!        0.000635
good value          0.000600
Great value.        0.000565
good deal           0.000565
Good deal           0.000529
OK                  0.000529
As expected         0.000529
Great product       0.000529
Great deal          0.000529
Good batteries      0.000494
Good value          0.000494
great deal          0.000459
Great value!        0.000459
Great batteries.    0.000459
So far so good      0.000459
Good price          0.000459
Work great          0.000424
Good buy            0.000424
very good           0.000388
good batteries      0.000388
As described        0.000388
Great deal!         0.000388
as expected         0.000388
Good value.   

In [129]:
# updating stop words and tokenizing the the data 
# interesting, so this is gensim's stop words, worked pretty nice 
STOPWORDS = set(STOPWORDS).union(set(['as', 'great', 'use', 'buy', 'product', 'best', 'love', 'good', ])) # will update as I go further along

def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

def clean(text):
    ''' Using some regex to clean up the text '''
    tokens = re.sub(r'[^a-zA-Z ^0-9]', '', text)

In [130]:
# Quick sample to be sure it works
tokenize('A Quick sample sentence to check tokenization process. If all goes well, this sentence as one one as should be coherent after tokenized.')

['quick',
 'sample',
 'sentence',
 'check',
 'tokenization',
 'process',
 'goes',
 'sentence',
 'coherent',
 'tokenized']

In [131]:
# Tokenizing the Reviews Text column into it's own new column 'Tokens'
df['tokens'] = df['reviews.text'].apply(tokenize, clean)
df['tokens'].head()

0    [order, item, bad, quality, missing, backup, s...
1               [bulk, expensive, way, products, like]
2                             [duracell, price, happy]
3              [work, brand, batteries, better, price]
4                    [batteries, long, lasting, price]
Name: tokens, dtype: object

In [132]:
df['tokens'].value_counts(normalize=True)[:50]

[]                     0.020789
[price]                0.008824
[batteries]            0.007306
[value]                0.006918
[work]                 0.005188
[deal]                 0.004412
[batteries, price]     0.003671
[ok]                   0.002753
[works]                0.002718
[excellent]            0.002471
[far]                  0.002047
[expected]             0.001835
[batteries, work]      0.001518
[long]                 0.001518
[price, batteries]     0.001412
[nice]                 0.001271
[quality]              0.001165
[described]            0.001094
[perfect]              0.001024
[advertised]           0.000988
[thanks]               0.000988
[battery]              0.000812
[thank]                0.000777
[work, price]          0.000777
[work, fine]           0.000777
[deal, batteries]      0.000741
[item]                 0.000706
[beat, price]          0.000671
[works, expected]      0.000635
[value, money]         0.000600
[quality, price]       0.000565
[price, 

## Fit a Gensim LDA Topic Model 

In [133]:
# A Dictionary to represent all the words in the corpus
id2word = corpora.Dictionary(df['tokens'])

In [134]:
# checking to be sure it worked
id2word.token2id['electric'], id2word[768]

(761, 'bht')

In [135]:
id2word.doc2bow(tokenize('Something Something Dark Side. Yo Yo, so this is my sentence, Yo.'))

[(2484, 3), (3787, 1)]

In [136]:
len(id2word.keys())

9614

In [137]:
# Now to remove some extreme Values from the Dictionary
# Will likely revist this right here depending on how accurate the LDA turns out, if it turns out well then may leave it
id2word.filter_extremes(no_below=.5, no_above=.95)

In [138]:
len(id2word.keys())

9614

In [139]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [140]:
corpus[69][:15]
''' So this translates into...im not sure exactly, revist this
    Either way, at this point of the process its ready LDA'''

' So this translates into...im not sure exactly, revist this\n    Either way, at this point of the process its ready LDA'

In [141]:
lda = LdaMulticore(corpus = corpus,
                   id2word = id2word,
                   random_state = 69, 
                   num_topics = 15, 
                   passes = 10, # be weary, over doing it here can reallllllly slow things down
                   workers = 8)

In [142]:
lda.print_topics()

[(0,
  '0.044*"kids" + 0.040*"tablet" + 0.029*"recommend" + 0.029*"kindle" + 0.021*"easy" + 0.018*"games" + 0.015*"play" + 0.012*"apps" + 0.011*"highly" + 0.011*"case"'),
 (1,
  '0.031*"batteries" + 0.020*"expensive" + 0.017*"like" + 0.016*"amazon" + 0.009*"thank" + 0.008*"things" + 0.007*"new" + 0.007*"camera" + 0.007*"long" + 0.006*"ve"'),
 (2,
  '0.019*"amazon" + 0.015*"bought" + 0.014*"purchase" + 0.012*"tablet" + 0.010*"echo" + 0.009*"alexa" + 0.009*"happy" + 0.008*"sound" + 0.008*"device" + 0.008*"tap"'),
 (3,
  '0.030*"screen" + 0.022*"tablet" + 0.016*"device" + 0.015*"value" + 0.014*"reading" + 0.013*"easy" + 0.013*"books" + 0.013*"light" + 0.012*"like" + 0.010*"apps"'),
 (4,
  '0.043*"easy" + 0.032*"battery" + 0.029*"tablet" + 0.028*"kindle" + 0.020*"charge" + 0.019*"life" + 0.017*"price" + 0.014*"read" + 0.014*"fast" + 0.009*"light"'),
 (5,
  '0.056*"quality" + 0.056*"price" + 0.029*"like" + 0.021*"far" + 0.018*"amazon" + 0.017*"better" + 0.014*"beat" + 0.014*"size" + 0.010*"

In [143]:
''' Cleaning up that above jumply gook'''
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:5]) for t in words]

for id, t in enumerate(topics):
    print(f"----- Topic {id} -----")
    print(t)
    print('\n')
    
    '''  SO already after just seeing these results and not even seeing the Topic Distance Visulaization
         it looks like there were too many topics, ie tablet is mentioned in several topics
    '''

----- Topic 0 -----
kids tablet recommend kindle easy


----- Topic 1 -----
batteries expensive like amazon thank


----- Topic 2 -----
amazon bought purchase tablet echo


----- Topic 3 -----
screen tablet device value reading


----- Topic 4 -----
easy battery tablet kindle charge


----- Topic 5 -----
quality price like far amazon


----- Topic 6 -----
kindle gift bought christmas price


----- Topic 7 -----
amazon batteries need tablet easy


----- Topic 8 -----
batteries long price work brand


----- Topic 9 -----
loves old tablet year bought


----- Topic 10 -----
tablet kids set time parental


----- Topic 11 -----
tablet apps kids amazon like


----- Topic 12 -----
amazon time bought price prime


----- Topic 13 -----
tablet kindle loves books easy


----- Topic 14 -----
price works nice expected item




In [144]:
''' Gonna check out the Visualization from lecture, and go from there to confirm my suspicions '''

pyLDAvis.enable_notebook()

In [145]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/Users/Inceptive/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.272438  0.055435       1        1  18.307318
9      0.198102  0.075305       2        1  10.908167
6      0.056705 -0.137298       3        1   7.344400
11     0.071947  0.090128       4        1   6.820554
3      0.059487 -0.075056       5        1   6.644115
0      0.157972  0.019310       6        1   6.581722
2     -0.085732  0.010239       7        1   5.797801
13     0.135681 -0.030595       8        1   5.769108
4      0.024270 -0.105635       9        1   5.688893
12    -0.072601  0.054760      10        1   4.800375
10     0.106132  0.123528      11        1   4.750304
5     -0.088055 -0.079038      12        1   4.544633
7     -0.090981  0.052379      13        1   4.271327
14    -0.047396 -0.085342      14        1   4.233976
1     -0.153094  0.031880      15        1   3.537305, topic_info=     Category         Freq       Term        Total  loglift  logprob
19    Default  7652.000000  batteries  7652.000000  30.0000  30.0000
4189  Default  6761.000000     tablet  6761.000000  29.0000  29.0000
18    Default  4988.000000      price  4988.000000  28.0000  28.0000
2609  Default  2077.000000      loves  2077.000000  27.0000  27.0000
94    Default  1417.000000      works  1417.000000  26.0000  26.0000
196   Default  2460.000000       kids  2460.000000  25.0000  25.0000
8     Default  1285.000000    quality  1285.000000  24.0000  24.0000
598   Default  2031.000000        old  2031.000000  23.0000  23.0000
103   Default  1694.000000       year  1694.000000  22.0000  22.0000
895   Default  2900.000000       easy  2900.000000  21.0000  21.0000
24    Default  3767.000000     amazon  3767.000000  20.0000  20.0000
3149  Default  2920.000000     kindle  2920.000000  19.0000  19.0000
2043  Default  1804.000000       apps  1804.000000  18.0000  18.0000
3     Default  2045.000000    battery  2045.000000  17.0000  17.0000
21    Default  1302.000000      brand  1302.000000  16.0000  16.0000
1052  Default  1062.000000       gift  1062.000000  15.0000  15.0000
10    Default  2346.000000       work  2346.000000  14.0000  14.0000
23    Default  2461.000000       long  2461.000000  13.0000  13.0000
765   Default  1663.000000      games  1663.000000  12.0000  12.0000
169   Default  1029.000000  recommend  1029.000000  11.0000  11.0000
1608  Default   911.000000   daughter   911.000000  10.0000  10.0000
28    Default  3120.000000     bought  3120.000000   9.0000   9.0000
2238  Default  1582.000000     screen  1582.000000   8.0000   8.0000
13    Default  2440.000000       like  2440.000000   7.0000   7.0000
73    Default   879.000000     brands   879.000000   6.0000   6.0000
30    Default   879.000000  christmas   879.000000   5.0000   5.0000
12    Default   482.000000  expensive   482.000000   4.0000   4.0000
1138  Default  1163.000000    reading  1163.000000   3.0000   3.0000
263   Default   343.000000   expected   343.000000   2.0000   2.0000
224   Default   548.000000     charge   548.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
557   Topic15    50.757698     couple   193.749420   2.0023  -5.3089
155   Topic15    43.289440       xbox   162.307785   2.0202  -5.4680
105   Topic15    86.992416     things   535.271179   1.5249  -4.7701
1241  Topic15    55.545662       wish   274.518158   1.7440  -5.2187
675   Topic15    67.648773     camera   396.276520   1.5740  -5.0216
944   Topic15    24.302084      stick    72.629448   2.2470  -6.0454
13    Topic15   170.334518       like  2440.918213   0.6794  -4.0982
19    Topic15   315.986603  batteries  7652.542969   0.1547  -3.4802
465   Topic15    41.408871    remotes   218.348648   1.6792  -5.5125
14    Topic15    49.405869   products   327.408569   1.4507  -5.3359
24    Topic15   165.459290     amazon  3767.341553   0.2164  -4.1272
3093  Topic15    17.661882   improved    46.211235   2.3800  -6.3645
413

## Select Appropriate Number of Topics

In [146]:
id2word = corpora.Dictionary(df['tokens'])
id2word.filter_extremes(no_below=.075, no_above=.925)
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [147]:
''' So upon the results of the above LDA, going to tweak some parameters on this LDA test
    here until everything looks right '''

lda2 = LdaMulticore(corpus=corpus,
                    id2word = id2word,
                    random_state=69,
                    num_topics= 2, # This ended up being two
                    passes = 20, 
                    workers = 6)
lda2.print_topics()

[(0,
  '0.041*"tablet" + 0.017*"easy" + 0.017*"kindle" + 0.016*"bought" + 0.014*"kids" + 0.013*"loves" + 0.013*"old" + 0.011*"apps" + 0.011*"year" + 0.011*"games"'),
 (1,
  '0.049*"batteries" + 0.035*"price" + 0.018*"long" + 0.017*"work" + 0.017*"amazon" + 0.017*"battery" + 0.010*"quality" + 0.009*"like" + 0.009*"value" + 0.008*"brand"')]

In [148]:
''' Cleaning up that above jumply gook'''
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda2.print_topics()]
topics = [' '.join(t[0:10]) for t in words]

for id, t in enumerate(topics):
    print(f"----- Topic {id} -----")
    print(t)
    print('\n')

----- Topic 0 -----
tablet easy kindle bought kids loves old apps year games


----- Topic 1 -----
batteries price long work amazon battery quality like value brand




## Create Some Dope Visualizations

In [149]:
''' For starts, the LDA visualization from lecture is epic!!!'''


' For starts, the LDA visualization from lecture is epic!!!'

In [150]:
''' Rerunning Visualizations with better optimized Topic Counts '''
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda2, corpus, id2word)

/Users/Inceptive/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.178459  0.0       1        1  55.517155
1     -0.178459  0.0       2        1  44.482838, topic_info=     Category         Freq         Term        Total  loglift  logprob
19    Default  6373.000000    batteries  6373.000000  30.0000  30.0000
4189  Default  6637.000000       tablet  6637.000000  29.0000  29.0000
18    Default  5291.000000        price  5291.000000  28.0000  28.0000
23    Default  2322.000000         long  2322.000000  27.0000  27.0000
10    Default  2224.000000         work  2224.000000  26.0000  26.0000
3149  Default  2781.000000       kindle  2781.000000  25.0000  25.0000
895   Default  2808.000000         easy  2808.000000  24.0000  24.0000
3     Default  2202.000000      battery  2202.000000  23.0000  23.0000
2609  Default  2126.000000        loves  2126.000000  22.0000  22.0000
598   Default  2104.000000          old  2104.000000  21.0000  21.0000
196   Default  2389.000000         kids  2389.000000  20.0000  20.0000
2043  Default  1788.000000         apps  1788.000000  19.0000  19.0000
765   Default  1697.000000        games  1697.000000  18.0000  18.0000
103   Default  1741.000000         year  1741.000000  17.0000  17.0000
21    Default  1081.000000        brand  1081.000000  16.0000  16.0000
2238  Default  1564.000000       screen  1564.000000  15.0000  15.0000
439   Default  1127.000000        value  1127.000000  14.0000  14.0000
3165  Default  1418.000000        books  1418.000000  13.0000  13.0000
8     Default  1470.000000      quality  1470.000000  12.0000  12.0000
462   Default  1108.000000         play  1108.000000  11.0000  11.0000
73    Default   744.000000       brands   744.000000  10.0000  10.0000
1138  Default  1127.000000      reading  1127.000000   9.0000   9.0000
1052  Default  1020.000000         gift  1020.000000   8.0000   8.0000
928   Default  1145.000000         read  1145.000000   7.0000   7.0000
1608  Default   923.000000     daughter   923.000000   6.0000   6.0000
58    Default  1084.000000         life  1084.000000   5.0000   5.0000
859   Default   914.000000          son   914.000000   4.0000   4.0000
495   Default   876.000000         deal   876.000000   3.0000   3.0000
171   Default   910.000000           ve   910.000000   2.0000   2.0000
28    Default  3009.000000       bought  3009.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
79     Topic2   461.157196       longer   471.547058   0.7878  -5.6339
14     Topic2   373.923859     products   382.106018   0.7884  -5.8436
59     Topic2   346.817841           ll   355.269562   0.7860  -5.9188
8      Topic2  1350.293335      quality  1470.557983   0.7247  -4.5596
18     Topic2  4480.576172        price  5291.477539   0.6437  -3.3601
495    Topic2   808.610657         deal   876.073303   0.7299  -5.0723
171    Topic2   830.973999           ve   910.585999   0.7186  -5.0450
34     Topic2   363.218750        haven   374.200226   0.7803  -5.8726
58     Topic2   954.509644         life  1084.970581   0.6820  -4.9064
44     Topic2   535.266174        cheap   576.386841   0.7361  -5.4849
228    Topic2   358.467682       worked   370.297974   0.7776  -5.8858
260    Topic2   398.335876  performance   416.734955   0.7649  -5.7804
72     Topic2   485.995911         beat   526.759094   0.7295  -5.5814
12     Topic2   467.085571    expensive   505.265320   0.7315  -5.6211
263    Topic2   375.284302     expected   392.517548   0.7652  -5.8400
271    Topic2   491.422333         fine   546.778748   0.7033  -5.5703
199    Topic2   955.251160          far  1240.389648   0.5489  -4.9057
63     Topic2   460.071167       buying   506.840576   0.7133  -5.6363
24     Topic2  2203.079834       amazon  3885.636719   0.2426  -4.0700
264    Topic2   586.818909        money   785.838379   0.5180  -5.3929
13     Topic2  1221.927124         like  2456.310059   0.1118  -4.659

In [126]:
''' So there two main overlaps, one consisting of three overlapping one area, another with two, so if those could 
    in essenece be brought from three to one & two to one, would have to subtract 3 from the topic counts, IN theory
'''

' So there two main overlaps, one consisting of three overlapping one area, another with two, so if those could \n    in essenece be brought from three to one & two to one, would have to subtract 3 from the topic counts, IN theory\n'

In [ ]:
''' Upon some LDA visuals, gonna add some sTOP words: great, use, buy, product, best, love.
    I think that by removing those, Ill get even closer to the core of what the data is conveying here
'''

## Some notes on my findings

In [ ]:
'''
So the main Topics were... Batteries and Tablets. Im pretty blown away that NLP technqiues were able to sort through all that
Text in the original corpus of Amazon Reviews, and distill it down to just these main topics in a nice easy to understand way.
I'm pretty stunned at the moment lol
'''

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling